In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hmmlearn  

     |████████████████████████████████| 374 kB 5.4 MB/s 


In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM, GMMHMM
from sklearn.model_selection import train_test_split
from tqdm import tqdm 

In [ ]:
import warnings
import logging
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from docopt import docopt

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/S7 Project/BTC.csv')
data = data[4278:47892]

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

In [ ]:
# extracting features
def extract_features(data):
    open_price = np.array(data['Open'])
    close_price = np.array(data['Close'])
    high_price = np.array(data['High'])
    low_price = np.array(data['Low'])
    frac_change = (close_price - open_price) / open_price
    frac_high = (high_price - open_price) / open_price
    frac_low = (open_price - low_price) / open_price
    return np.column_stack((frac_change, frac_high, frac_low))


In [ ]:
def get_most_probable_outcome(day_index):
    previous_data_start_index = max(0, day_index - 20)
    previous_data_end_index = max(0, day_index - 1)
    previous_data = test_data.iloc[previous_data_end_index: previous_data_start_index]
    previous_data_features = extract_features(previous_data)

    outcome_score = []
    for possible_outcome in possible_outcomes:
        total_data = np.row_stack(
            (previous_data_features, possible_outcome))
        outcome_score.append(model.score(total_data))
    most_probable_outcome = possible_outcomes[np.argmax(
        outcome_score)]

    return most_probable_outcome

In [ ]:
#All possible outcomes
frac_change_range = np.linspace(-0.1, 0.1, 60)
frac_high_range = np.linspace(0, 0.1, 10)
frac_low_range = np.linspace(0, 0.1, 10)
possible_outcomes = np.array(list(itertools.product(
    frac_change_range, frac_high_range, frac_low_range)))


In [ ]:
model = GMMHMM(n_components=4, n_mix=5)
model.fit(extract_features(data))

GMMHMM(covars_prior=array([[[-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5]],

       [[-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5, -1.5],
        [-1.5, -1.5...
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]]),
       means_weight=array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]]),
       n_compone

In [ ]:
predicted = []
for day_index in tqdm(range(len(test_data))):
  open_price = test_data.iloc[day_index]['Open']
  predicted_frac_change, _, _ = get_most_probable_outcome(day_index)
  open_price * (1 + predicted_frac_change)
  predicted.append(open_price * (1 + predicted_frac_change))
with open('/content/drive/MyDrive/project/checking.txt', 'w') as filehandle:
  for listitem in predicted:
    filehandle.write(str(listitem)+", ")


100%|██████████| 700/700 [1:01:48<00:00,  5.30s/it]


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/S7 Project/BTC.csv')
data = data[4278:47892]
close = data.Close
train,test = train_test_split(
            close, test_size=0.2, shuffle=False)
actual=test.tolist()

In [ ]:

from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(predicted, actual , squared=False)
print("rmse: " +str(rmse))

mape=0
for i in range(len(actual)):
  mape+=abs(actual[i]-predicted[i])/actual[i]
mape/=len(actual)
print("mape: " +str(mape))


absolute_prediction_errors=[]
for i in range(len(actual)):
  absolute_prediction_errors.append(abs(actual[i]-predicted[i]))
absolute_prediction_errors = sorted(absolute_prediction_errors)
mean = sum(absolute_prediction_errors) / len(absolute_prediction_errors)
sd = 0
for i in absolute_prediction_errors:
  sd += (i-mean)**2
sd/=len(absolute_prediction_errors)
sd=pow(sd,0.5)
minimum = absolute_prediction_errors[0]
a25 = absolute_prediction_errors[(int)(25*len(absolute_prediction_errors))//100]
a50 = absolute_prediction_errors[(int)(50*len(absolute_prediction_errors))//100]
a75 = absolute_prediction_errors[(int)(75*len(absolute_prediction_errors))//100]
a95 = absolute_prediction_errors[(int)(95*len(absolute_prediction_errors))//100]
a97 = absolute_prediction_errors[(int) (97.5*len(absolute_prediction_errors))//100]
maximum = absolute_prediction_errors[-1]

print(mean)
print(sd)
print(minimum)
print(a25)
print(a50)
print(a75)
print(a95)
print(a97)
print(maximum)